# Análisis y visualización de datos con Python
# 5. Remodelación de datos


---

Una vez que los datos han sido limpiados y estructurados, la siguiente etapa en el proceso de análisis es la **manipulación y transformación de datos**. En esta fase, reorganizamos la información para que sea adecuada para análisis más avanzados o para la creación de visualizaciones. La manipulación de datos va más allá de la simple limpieza; se centra en la agregación, la combinación de conjuntos de datos y la reestructuración de la información.

En este notebook, continuaremos trabajando con el conjunto de datos de muertes asociadas a causas respiratorias, que ya hemos seleccionado y limpiado a partir del EDR del INEGI en los ejercicios anteriores. Nos enfocaremos en transformar este conjunto de datos para explorar patrones de mortalidad por causas respiratorias en 2023 desde diferentes perspectivas.

Los temas que exploraremos son:
* **Tablas pivote**: Crearemos tablas resumen para analizar datos con múltiples variables categóricas.
* **Combinación de DataFrames**: Uniremos información de múltiples tablas usando `merge()` y `concat()`.
* **Agregación de datos**: Utilizaremos `groupby()` para calcular estadísticas resumidas (como el conteo de muertes, promedios de edad, etc.) para diferentes grupos de interés.

Para empezar, cargaremos el conjunto de datos limpio que guardamos en formato Pickle, ya que este formato conserva los tipos de datos y la estructura del DataFrame, lo que nos ahorrará tiempo.

In [1]:
from dbfread import DBF
import pandas as pd
import numpy as np

df_respiratorio = pd.read_pickle('data_clean/defunciones_respiratorias_2023.pkl')
df_respiratorio.tail()

,SEXO,EDAD,EDAD_AGRU,ENT_OCURR,ENT_OCURR_NOM,MUN_OCURR,MUN_OCURR_NOM,AREA_UR,FECHA_OCURR,DIA_OCURR,MES_OCURR,ANIO_OCUR,DIA_SEMANA_OCURR,CAUSA_DEF,CAUSA_DEF_NOM,CAUSA_DEF_CLAS,TIPO_DEFUN,SITIO_OCUR,COND_CERT,DERECHOHAB
104329,Hombre,0.0,Menores de un año,32,Zacatecas,056,Zacatecas,Rural,2023-03-11,11.0,3.0,2023,Sábado,P251,Neumotórax originado en el período perinatal,"Neonatal hypoxia, aspiration, neonatal pneumonia",Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS
104330,Hombre,0.0,Menores de un año,32,Zacatecas,055,Villanueva,Rural,2023-06-19,19.0,6.0,2023,Lunes,P209,"Hipoxia intrauterina, no especificada","Neonatal hypoxia, aspiration, neonatal pneumonia",Enfermedad (Muerte natural),IMSS BIENESTAR,Médico tratante,Ninguna
104331,Mujer,0.0,Menores de un año,32,Zacatecas,024,Loreto,Urbana,2023-12-22,22.0,12.0,2023,Viernes,P249,"Síndrome de aspiración neonatal, sin otra espe...","Neonatal hypoxia, aspiration, neonatal pneumonia",Enfermedad (Muerte natural),Hogar,Otro médico,No especificada
104332,Mujer,0.0,Menores de un año,32,Zacatecas,056,Zacatecas,Urbana,2023-08-09,9.0,8.0,2023,Miércoles,P220,Síndrome de dificultad respiratoria del recién...,"Neonatal hypoxia, aspiration, neonatal pneumonia",Enfermedad (Muerte natural),IMSS,Médico tratante,IMSS
104333,Hombre,0.0,Menores de un año,32,Zacatecas,038,Pinos,Rural,2023-04-05,5.0,4.0,2023,Miércoles,P285,Insuficiencia respiratoria del recién nacido,"Neonatal hypoxia, aspiration, neonatal pneumonia",Enfermedad (Muerte natural),IMSS BIENESTAR,Otro médico,Ninguna


## 5.a Tabla pivote: relacionar variables

Una `tabla pivote` dinámica permite reorganizar y resumir datos de forma que faciliten su análisis al relacionar dos o más variables. Son útiles para calcular estadísticas resumidas como sumas, medias o conteos agrupados por diferentes categorías. 
Pare realizar una tabla pivote es necesario que el conjunto de datos se encuentre **ordenado** cuando, es decir, cada variable forma una columna y cada observación forma una fila.
Esta función se encuentra disponible en muchas hojas de cálculo, en `pandas` usaremos la función `pd.pivot_table()`.

La estructura típica de una tabla pivote se compone de las siguientes partes:
* **Filas (índice)**: las categorías principales por las que se agrupa el conjunto de datos.
* **Columnas**: las variables o categorías adicionales a relacionar y por las cuales se distribuyen los datos.
* **Valores**: los datos que se resumen en la intersección entre filas y columnas. Estos valores pueden ser sumatorias, promedios, conteos o cualquier otra función de agregación aplicada a los datos originales.
* **Campos adicionales** (opcional): algunas tablas pivote incluyen filtros adicionales para permitir una segmentación dinámica de los datos, como seleccionar solo un rango de fechas o una categoría específica.

Por ejemplo, supongamos que se quiere ver la relación entre el tipo de área de la localidad (urbana o rural)  y el sexo del fallecido. Lo primero que se debe de hacer es imaginar cómo se verá la tabla, colocando las filas, columnas y valores.


|        | Urbana | Rural | No esp |
|--------|------|------|------|
| Mujer  | ###  | ###  | ###  |
| Hombre | ###  | ###  | ###  |
| No esp | ###  | ###  | ###  |

donde '###'  representa es el número de ingresos de ese tipo de restos en el año

A continuación es necesario determinar donde está la información en el conjunto de datos ordenado o si no existe calcularla.

* Filas:  `SEXO`
* Columnas: `AREA_UR`
* Función: función `size()`

Basándonos en esto podemos escribir la función correspondiente.

**Nota**: La función `count()` es muy similar a `size()`, pero se aplica a cada columna y no cuenta los `nan`, por lo que los valores pueden variar.

In [2]:
# tabla pivote
pd.pivot_table(df_respiratorio, 
               index='SEXO', 
               columns='AREA_UR', 
               aggfunc='size'
               )

/tmp/ipykernel_20351/3421684332.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(df_respiratorio,


AREA_UR,No especificada,Rural,Urbana
SEXO,,,
Hombre,1240,10362,50348
Mujer,376,7176,34816
No especificado,4,4,8


Es posible generar tablas pivote mucho más complicadas a partir de un conjunto de datos ordenados. Por ejemplo, se puede calcular la edad promedio de cada uno de los grupos anteriores. En este caso necesitamos definir sobre que variable y con que función calcularemos el valor de las celdas de la tabla pivote.

* Filas:  `SEXO`
* Columnas: `AREA_UR`
* Valores: `EDAD`
* Función: `mean()`

In [3]:
# tabla pivote
pd.pivot_table(df_respiratorio, 
               index  =  'SEXO', 
               columns = 'AREA_UR', 
               values  = 'EDAD', # <- variable sobre la que se a a calcular 
               aggfunc = 'mean', # <- la operación
               )

/tmp/ipykernel_20351/2749300926.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(df_respiratorio,


AREA_UR,No especificada,Rural,Urbana
SEXO,,,
Hombre,49.620265,60.431467,59.549695
Mujer,52.495726,63.834704,66.127564
No especificado,NaN,19.500000,0.000000


Supongamos que nos interesa saber la relación entre el tipo de área (urbana o rural), el sexo y si es menor de edad, adulto o adulto mayor (60+ años). La tabla se vería:

|        | Urbana | Rural | No esp |
|--------|------|------|------|
| Mujer menor  | ###  | ###  | ###  |
| Mujer adulta | ###  | ###  | ###  |
| Mujer mayor  | ###  | ###  | ###  |
| Hombre menor | ###  | ###  | ###  |
| Hombre adulto| ###  | ###  | ###  |
| Hombre mayor | ###  | ###  | ###  |
| No esp menor | ###  | ###  | ###  |
| No esp adulto| ###  | ###  | ###  |
| No esp mayor | ###  | ###  | ###  |

A continuación es necesario determinar donde está la información en el conjunto de datos ordenado. Si no existe directamente se puede calcular

* Filas: `SEXO` y `EDAD` (clasificar)
* Columnas: `AREA_UR`
* Función: `size`

Se pueden usar varios variables o columnas para generar la tabla pivote. Esto genera un [multi-índice](https://pandas.pydata.org/docs/user_guide/advanced.html#). Podemos volver el multi-índice en columnas con la función `.reset_index()`.

In [4]:
# Función para clásificar por edad
f_edad_categoria = lambda edad: "Menor" if edad < 18 else ("Mayor" if edad > 60 else "Adulto")
# Creamos nueva columna con clasificación de edad
df_respiratorio['EDAD_CLASS'] = df_respiratorio['EDAD'].apply(f_edad_categoria)
display( df_respiratorio['EDAD_CLASS'].value_counts() )

# Tabla pivote
pd.pivot_table(df_respiratorio, 
               index   = ['SEXO','EDAD_CLASS'],  # <- multiples variables
               columns = 'AREA_UR', 
               aggfunc = 'size'
              )

EDAD_CLASS
Mayor     65516
Adulto    29617
Menor      9201
Name: count, dtype: int64

/tmp/ipykernel_20351/2277002635.py:8: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(df_respiratorio,


AREA_UR                     No especificada  Rural  Urbana
SEXO            EDAD_CLASS                                
Hombre          Adulto                  797   2567   17503
                Mayor                   349   6479   28963
                Menor                    94   1316    3882
Mujer           Adulto                  164   1257    7325
                Mayor                   164   4938   24622
                Menor                    48    981    2869
No especificado Adulto                    4      0       0
                Mayor                     0      1       0
                Menor                     0      3       8

Existen una gran cantidad de funciones que se pueden usar. Es recomendable revisar la documentación de [funciones de agrupamiento](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#seriesgroupby-computations-descriptive-stats).

Algunas funciones útiles son: 
* `mean`: promedio
* `median`: mediana
* `std`: desviación estándar
* `sum`: suma de valores
* `size`: tamaño del grupo
* `count`: conteo del grupo
* `first`: primer valor
* `last`: último valor
* `nth`: n-esimo valor
* `min`: valor mímino
* `max`: valor máximo

Además, se pueden usar funciones de otras bibliotecas como `numpy` o definir funciones especiales.

Por ejemplo, la siguiente función regresa los `strings` más comunes de una columna.

In [5]:
def textos_mas_comunes(series, n=5, sep=', '):
    textos = series.value_counts().head(n)
    textos = textos.index
    textos = sep.join(textos)
    return textos

pd.pivot_table(df_respiratorio, 
               index  =  'SEXO', 
               columns = 'AREA_UR', 
               values  = 'CAUSA_DEF_CLAS',
               aggfunc = textos_mas_comunes
              )

/tmp/ipykernel_20351/2045137328.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(df_respiratorio,


AREA_UR,No especificada,Rural,Urbana
SEXO,,,
Hombre,"Influenza and pneumonia, Drowning and suffocat...","Influenza and pneumonia, COPD, emphysema, BC, ...","Influenza and pneumonia, COPD, emphysema, BC, ..."
Mujer,"Influenza and pneumonia, Drowning and suffocat...","COPD, emphysema, BC, Influenza and pneumonia, ...","Influenza and pneumonia, COPD, emphysema, BC, ..."
No especificado,"Drowning and suffocation, HIV and associated i...","Neonatal hypoxia, aspiration, neonatal pneumon...","Neonatal hypoxia, aspiration, neonatal pneumon..."


### Normalización

Es posible usar una tabla como base de otras operaciones. Por ejemplo, se pueden agregar los totales de filas y columnas.

In [6]:
# tabla pivote guardar
pt_sexo_area = pd.pivot_table( df_respiratorio,  index='SEXO',  columns='AREA_UR', aggfunc='size' )

# suma de filas
total_filas = pt_sexo_area.sum(axis=1)
pt_sexo_area['Total SEXO'] = total_filas

#suma de columnas
total_columnas = pt_sexo_area.sum()
# Unir filas al final de la tabla
total_columnas = total_columnas.rename('Total AREA_UR').to_frame().T #formato
pt_sexo_area = pd.concat([pt_sexo_area,total_columnas]) #unir
# Mostrar
pt_sexo_area

/tmp/ipykernel_20351/1085409952.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pt_sexo_area = pd.pivot_table( df_respiratorio,  index='SEXO',  columns='AREA_UR', aggfunc='size' )


AREA_UR,No especificada,Rural,Urbana,Total SEXO
Hombre,1240,10362,50348,61950
Mujer,376,7176,34816,42368
No especificado,4,4,8,16
Total AREA_UR,1620,17542,85172,104334


La tabla pivote también permite normalizar los datos, calculando proporciones en lugar de conteos absolutos. Esto es útil para comparar distribuciones entre diferentes categorías. Puedes **normalizar** la tabla dividiendo por el total de la población, por el total de cada fila o por el total de cada columna.

Ejemplo 1: Para normalizar la tabla por el **total de la población**, se divide cada celda por el número total de defunciones, que es el tamaño del DataFrame original.

In [7]:
pt_sexo_area.divide( df_respiratorio.shape[0] )

AREA_UR,No especificada,Rural,Urbana,Total SEXO
Hombre,0.011885,0.099316,0.482566,0.593766
Mujer,0.003604,0.068779,0.333698,0.406080
No especificado,0.000038,0.000038,0.000077,0.000153
Total AREA_UR,0.015527,0.168133,0.816340,1.000000


Para normalizar por el **total de cada fila**, se usa `.div` para dividir por el total de las filas. Esto es útil para ver la distribución de áreas de defunción dentro de cada grupo de sexo.

In [8]:
pt_sexo_area.div( pt_sexo_area['Total SEXO'], axis=0 )

AREA_UR,No especificada,Rural,Urbana,Total SEXO
Hombre,0.020016,0.167264,0.812720,1.0
Mujer,0.008875,0.169373,0.821752,1.0
No especificado,0.250000,0.250000,0.500000,1.0
Total AREA_UR,0.015527,0.168133,0.816340,1.0


Para normalizar por el total de cada columna, se usa `.div()` con el parámetro `axis=0` para dividir por el total de las columnas. Esto es útil para ver la distribución de sexo dentro de cada tipo de área de defunción.

In [9]:
pt_sexo_area.divide( pt_sexo_area.loc['Total AREA_UR'], axis=1 )

AREA_UR,No especificada,Rural,Urbana,Total SEXO
Hombre,0.765432,0.590697,0.591133,0.593766
Mujer,0.232099,0.409075,0.408773,0.406080
No especificado,0.002469,0.000228,0.000094,0.000153
Total AREA_UR,1.000000,1.000000,1.000000,1.000000


## 5.b Melt: transformar a tidy data

Una de las herramientas más usadas para el análisis de datos es Excel, ya que es muy potente y fácil de usar. Sin embargo, muchas veces al diseñar las tablas esto se hace mezclando la obtención, limpieza y análisis de datos en un solo paso, lo cual puede dificultar su posterior manejo. Es común que las tablas de Excel no estén en formato tidy data, sino que estén en alguna variación similar a una tabla pivote. En ese caso es necesario reestructurar la tabla a formato tidy data.

Es importante destacar que existen muchos formatos de datos desordenados, por lo que puede ser necesario realizar una serie de correcciones manuales y automáticas dependiendo de la situación. El artículo original de [tidy data](http://vita.had.co.nz/papers/tidy-data.pdf) cubre varios ejemplos comunes. 
AIs generativas como ChatGPT y DeepSeek también pueden servir para obtener el código para reestructurar los datos, pero es necesario ser cuidadoso al escribir el prompt y revisar a detalle los resultados.

La función `pd.melt()` es útil para convertir datos en formato ancho (_wide_) como el de una tabla pivote a formato largo (_long_) cómo el de tidy data. Cuando usamos `pd.melt()`, seleccionamos una o más columnas como identificadores y convertimos las demás columnas en dos columnas nuevas: una para los nombres de las variables y otra para sus valores.

La función `melt` tiene los siguientes parámetros clave:
- `id_vars`: Columnas que actúan como identificadores y no deben transformarse.
- `value_vars`: Columnas que se convertirán en las nuevas columnas de variable(s) y valor. Si no se especifica, se utilizan todas las columnas que no están en `id_vars`.
- `var_name`: Nombre personalizado para la columna de variables.
- `value_name`: Nombre personalizado para la columna de valores.

Por ejemplo, supongamos que se quiere transformar la tabla pivote `pt_sexo_area` a una tabla en formato tidy data.

In [10]:
pt_sexo_area

AREA_UR,No especificada,Rural,Urbana,Total SEXO
Hombre,1240,10362,50348,61950
Mujer,376,7176,34816,42368
No especificado,4,4,8,16
Total AREA_UR,1620,17542,85172,104334


El primer paso es definir cuáles son las variables y cuáles son las observaciones.

* Variables: Sexo, área urbana
* Observación: conteo de decesos por grupo

A continuación se determina que columnas es necesario reestructurar y cuáles no,

* No transformar (`id_vars`): `index`.
* Transformar (`value_vars`): `No especificada`, `Rural`, `Urbana`.

También es buen momento de decidir un nombre para la columna del valor, en este caso la regresaremos a su nombre inicial `AREA_UR`

Antes de realizar la operación de `pd.melt()` es importante verificar si se necesita alguna otra limpieza. En este caso es necesario:
* Quitar columna y fila de total
* Volver el índice en columna

Basándonos en esto podemos escribir la función correspondiente. 

In [11]:
# Operaciones de limpieza
df_sexoarea_conteo = pt_sexo_area.drop(index=['Total AREA_UR'], columns=['Total SEXO']) \
                .reset_index().rename(columns={'index':'SEXO'})
display( df_sexoarea_conteo )

# Melt transformar de pivote a tabla ordenada
df_sexoarea_conteo = df_sexoarea_conteo.melt(
                id_vars='SEXO',
                value_name='conteo',
                value_vars= ['No especificada', 'Rural', 'Urbana'],
                )
df_sexoarea_conteo

AREA_UR,SEXO,No especificada,Rural,Urbana
0,Hombre,1240,10362,50348
1,Mujer,376,7176,34816
2,No especificado,4,4,8


,SEXO,AREA_UR,conteo
0,Hombre,No especificada,1240
1,Mujer,No especificada,376
2,No especificado,No especificada,4
3,Hombre,Rural,10362
4,Mujer,Rural,7176
5,No especificado,Rural,4
6,Hombre,Urbana,50348
7,Mujer,Urbana,34816
8,No especificado,Urbana,8


## 5.c Concat: Unir tablas uno a uno

Es posible unir tablas de distintas maneras usando pandas, en este tutorial veremos solo las más sencillas, pero les recomendamos ver la guia de [Merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

Por ejemplo, estas series muestran la cantidad de ingresos por tipo de restos y sexo para menores y mayores de edad.

In [12]:
df_sexoarea_minedad = df_respiratorio.loc[df_respiratorio['EDAD']<18, ['SEXO','AREA_UR']] \
                          .value_counts().rename('N_menores_edad').reset_index()
df_sexoarea_minedad

,SEXO,AREA_UR,N_menores_edad
0,Hombre,Urbana,3882
1,Mujer,Urbana,2869
2,Hombre,Rural,1316
3,Mujer,Rural,981
4,Hombre,No especificada,94
5,Mujer,No especificada,48
6,No especificado,Urbana,8
7,No especificado,Rural,3


In [13]:
df_sexoarea_maxedad = df_respiratorio.loc[df_respiratorio['EDAD']>=18, ['SEXO','AREA_UR']] \
                          .value_counts().rename('N_mayores_edad').reset_index()
df_sexoarea_maxedad

,SEXO,AREA_UR,N_mayores_edad
0,Hombre,Urbana,46435
1,Mujer,Urbana,31937
2,Hombre,Rural,9044
3,Mujer,Rural,6194
4,Hombre,No especificada,962
5,Mujer,No especificada,303
6,No especificado,Rural,1


Una forma de unir dos tablas es con la función `concat()`. Esta función toma una lista de dataframes y las une por el índice o las columnas. En este caso es una unión uno a uno, es decir, cada valor del índice o columna es único y no se repiten.
Es importante notar que, aunque los valores no se repiten, los índices sí se repiten, por ejemplo ('Feto', 'Masculino') aparece dos veces.
Esta operación puede ser vista como apilar las tablas a lo alto (filas), uniendo a lo ancho (columnas).

In [14]:
pd.concat([df_sexoarea_minedad,df_sexoarea_maxedad])

,SEXO,AREA_UR,N_menores_edad,N_mayores_edad
0,Hombre,Urbana,3882.0,NaN
1,Mujer,Urbana,2869.0,NaN
2,Hombre,Rural,1316.0,NaN
3,Mujer,Rural,981.0,NaN
4,Hombre,No especificada,94.0,NaN
5,Mujer,No especificada,48.0,NaN
6,No especificado,Urbana,8.0,NaN
7,No especificado,Rural,3.0,NaN
0,Hombre,Urbana,NaN,46435.0
1,Mujer,Urbana,NaN,31937.0


En segundo lugar, uniremos usando `pd.concat(axis=1)`, es decir, los valores se unirán en las filas.
Nota como los nombres de las columnas se mantuvieron como estaban en las tablas originales.
Esta operación puede ser vista como pegar las tablas a lo ancho (columnas), uniendo a lo alto (filas).

In [15]:
pd.concat([df_sexoarea_minedad,df_sexoarea_maxedad], axis=1)

,SEXO,AREA_UR,N_menores_edad,SEXO,AREA_UR,N_mayores_edad
0,Hombre,Urbana,3882,Hombre,Urbana,46435.0
1,Mujer,Urbana,2869,Mujer,Urbana,31937.0
2,Hombre,Rural,1316,Hombre,Rural,9044.0
3,Mujer,Rural,981,Mujer,Rural,6194.0
4,Hombre,No especificada,94,Hombre,No especificada,962.0
5,Mujer,No especificada,48,Mujer,No especificada,303.0
6,No especificado,Urbana,8,No especificado,Rural,1.0
7,No especificado,Rural,3,NaN,NaN,NaN


## 5.d Merge: Unir tablas muchos a uno

Otro caso de unir tablas es cuando se tiene una relación de **muchos a uno**. Esto ocurre cuando múltiples filas en una tabla se corresponden con una única fila en otra tabla. En este contexto, unimos un conjunto de datos primario con un catálogo o una tabla de referencia.

Por ejemplo, recordemos el mapeo del catálogo de entidades y municipios. Originalmente, el EDR contiene los códigos correspondientes a los diferentes niveles geográficos en las variables `ENT_OCURR` y `MUN_OCURR`. Tomaremos una muestra de estas columnas y la guardaremos en `data`.

In [16]:
data = df_respiratorio[['ENT_OCURR', 'MUN_OCURR']].sample(10)
data

,ENT_OCURR,MUN_OCURR
31494,09,007
85521,25,001
88966,26,030
21941,08,019
4511,15,074
13077,02,001
36273,10,007
7605,26,030
52740,15,054
59292,15,031


In [17]:
data = df_respiratorio[['ENT_OCURR', 'MUN_OCURR']].sample(10)
data

,ENT_OCURR,MUN_OCURR
80689,22,011
12667,02,004
20874,07,070
14023,04,002
30604,09,012
15163,05,030
15233,05,018
14184,04,010
79811,21,114
55952,15,025


Por otro lado, tenemos `CATEMLDE23.dbf`, el cual contiene las claves geográficas y nombres del Marco Geoestadístico del INEGI.

In [18]:
# Cargar marco geoestadístico
file_path = './data_raw/defunciones_base_datos_2023_dbf/CATEMLDE23.dbf'
mapeo_lugar = DBF(file_path)
mapeo_lugar = pd.DataFrame(mapeo_lugar)

# Seleccionar solo entidades para generar tabla mapeo
mapeo_ent = mapeo_lugar.loc[mapeo_lugar['CVE_MUN']=='000', ['CVE_ENT','NOM_LOC']]
display(mapeo_ent.head())

,CVE_ENT,NOM_LOC
0,01,Aguascalientes
155,02,Baja California
293,03,Baja California Sur
360,04,Campeche
577,05,Coahuila de Zaragoza


Ahora, ambas tablas (`data` y `mapeo_ent`) comparten una columna con la clave de la entidad (`ENT_OCURR` en `data` y `CVE_ENT` en `mapeo_ent`), lo que permite unirlas en una sola. Para esto usamos la función `pd.merge()`.

Los parámetros principales de `pd.merge()` son:

  * **`left` y `right`**: Son los dos DataFrames que se van a unir.
  * **`on`**: La(s) columna(s) común(es) que se usa(n) para unir las tablas. Si las columnas tienen nombres diferentes en cada DataFrame, se usan los parámetros `left_on` y `right_on`.
  * **`how`**: Define el tipo de unión, similar a las operaciones de SQL.

El parámetro `how` acepta las siguientes opciones:

  * **`inner` (unión interna)**: El valor por defecto. Devuelve solo las filas que tienen un valor coincidente en la columna de unión en **ambos** DataFrames. Es el resultado de la intersección.
  * **`outer` (unión externa)**: Devuelve todas las filas de **ambos** DataFrames. Donde no hay coincidencias, se rellenan los valores con `NaN`.
  * **`left` (unión izquierda)**: Devuelve todas las filas del DataFrame `left` y las filas coincidentes del DataFrame `right`. Donde no hay coincidencia en la derecha, se llenan con `NaN`.
  * **`right` (unión derecha)**: Devuelve todas las filas del DataFrame `right` y las filas coincidentes del DataFrame `left`. Donde no hay coincidencia en la izquierda, se llenan con `NaN`.

Al unir las tablas, las columnas que no son parte de la clave de unión se incluyen en el resultado. Si hay columnas con el mismo nombre en ambos DataFrames (y no son la clave de unión), `pandas` añade automáticamente un sufijo (`_x` y `_y`) para evitar conflictos. Puedes personalizar estos sufijos con el parámetro `suffixes`.

Para nuestro ejemplo, usaremos la opción `how='left'`, ya que queremos conservar todos los registros de defunciones (`data`) y añadirles los nombres de las entidades correspondientes del catálogo (`mapeo_ent`).

In [19]:
pd.merge(
    left=data,
    right=mapeo_ent,
    left_on='ENT_OCURR',
    right_on='CVE_ENT',
    how='left'
)

,ENT_OCURR,MUN_OCURR,CVE_ENT,NOM_LOC
0,22,011,22,Querétaro
1,02,004,02,Baja California
2,07,070,07,Chiapas
3,04,002,04,Campeche
4,09,012,09,Ciudad de México
5,05,030,05,Coahuila de Zaragoza
6,05,018,05,Coahuila de Zaragoza
7,04,010,04,Campeche
8,21,114,21,Puebla
9,15,025,15,México


Existen muchas formas de unir tablas usando llaves, una vez más recomendamos ver el tutorial de [merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

## 5.e División-aplicación-combinación

El pipeline [split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html) es un enfoque muy común en el análisis de datos y se refiere a un proceso de tres pasos:
* _Split_ (divide): El conjunto de datos original se divide en grupos basados en una o más columnas. En primer lugar, se divide el conjunto de datos en grupos basados en una o varias columnas utilizando la función `groupby()`. Esto genera un objeto `GroupBy` que contiene los grupos y se puede aplicar una función o transformación a cada grupo por separado.
* _Apply_ (aplicación): Se aplica alguna función o transformación a cada grupo por separado. Luego, se aplica alguna función o transformación a cada grupo por separado. Esta función puede ser de varios tipos:
    * _Aggregate_ (agregar): calcular una estadística de resumen (o varias estadísticas) para cada grupo. Por ejemplo, calcular la suma o tamaño de los grupos.
    * _Transform_ (transformar): realizar algunos cálculos específicos del grupo y devolver un objeto con el mismo índice.  Por ejemplo, rellenar los valores faltantes (`nan`) dentro de un grupo con un valor derivado de ese mismo grupo.
    * _Filter_ (filtrar): descartar algunos grupos, según un cálculo por grupo que evalúa Verdadero o Falso. Por ejemplo, descartar datos que pertenecen a grupos con muy pocos miembros.
    * Una combinación de lo anterior
* _Combine_ (combinar): Los resultados de cada grupo se combinan en un único DataFrame utilizando la función `concat()`, `merge()` u otra función de combinación de pandas.

A continuación veremos una introducción a este pipeline, aunque se recomienda ver la documentación de [split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html).

### Agrupar por características

La función `groupby()` se utiliza para agrupar datos en función de una o varias columnas y aplicar una función de agregación a cada grupo.
Esta función divide la tabla o DataFrame en sub-tablas o grupos de acuerdo a los valores de una o más columnas. El resultado es un objeto similar a un diccionario, donde las llaves son los valores por los que se dividió la tabla y los valores las sub-tablas resultantes. Este objeto es iterable y se le pueden aplicar funciones por separado a cada grupo.


In [20]:
# Agrupar por columna
groups = df_respiratorio.groupby('SEXO', observed=True)
# Iterar por cada grupo
for key, data in groups:
    print(f"Grupo:{key}, \tTamaño:{data.shape} \tType':{type(data)}")

Grupo:Hombre, 	Tamaño:(61950, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:Mujer, 	Tamaño:(42368, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:No especificado, 	Tamaño:(16, 21) 	Type':<class 'pandas.core.frame.DataFrame'>


Usando varias columnas para agrupar se genera un `multiindex`.

Esto crea una agrupación jerárquica que considera las combinaciones posibles entre "SEXO" y "AREA_UR".

In [21]:
groups = df_respiratorio.groupby(['SEXO','AREA_UR'])

for key, data in groups:
    print(f"Grupo:{key}, \tTamaño:{data.shape} \tType':{type(data)}")

Grupo:('Hombre', 'No especificada'), 	Tamaño:(1240, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('Hombre', 'Rural'), 	Tamaño:(10362, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('Hombre', 'Urbana'), 	Tamaño:(50348, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('Mujer', 'No especificada'), 	Tamaño:(376, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('Mujer', 'Rural'), 	Tamaño:(7176, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('Mujer', 'Urbana'), 	Tamaño:(34816, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('No especificado', 'No especificada'), 	Tamaño:(4, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('No especificado', 'Rural'), 	Tamaño:(4, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:('No especificado', 'Urbana'), 	Tamaño:(8, 21) 	Type':<class 'pandas.core.frame.DataFrame'>


/tmp/ipykernel_20351/1828698035.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = df_respiratorio.groupby(['SEXO','AREA_UR'])


Para incluir los `nan` en los grupos se usa la opción `dropna=False`

In [22]:
groups = df_respiratorio.groupby('MES_OCURR', observed=False, dropna=False)

for key, data in groups:
    print(f"Grupo:{key}, \tTamaño:{data.shape} \tType':{type(data)}")

Grupo:1.0, 	Tamaño:(12007, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:2.0, 	Tamaño:(8825, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:3.0, 	Tamaño:(8750, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:4.0, 	Tamaño:(8217, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:5.0, 	Tamaño:(8170, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:6.0, 	Tamaño:(8693, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:7.0, 	Tamaño:(7816, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:8.0, 	Tamaño:(7761, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:9.0, 	Tamaño:(7796, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:10.0, 	Tamaño:(8092, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:11.0, 	Tamaño:(8641, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:12.0, 	Tamaño:(9549, 21) 	Type':<class 'pandas.core.frame.DataFrame'>
Grupo:nan, 	Tamaño:(17, 21) 	Type':<class 'pandas.core.frame.DataFrame'>


### Agregar

Las funciones de agregación son funciones en Pandas que se utilizan en conjunto con la función `groupby()` para realizar cálculos resumidos sobre un conjunto de datos agrupados. Estas funciones toman un conjunto de valores y los resumen en un solo valor. Estas son las funciones que vimos anteriormente en las tablas pivote, las puedes consultar en [funciones de agrupamiento](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#seriesgroupby-computations-descriptive-stats).


In [23]:
df_respiratorio.groupby(by=['SEXO','AREA_UR']).size()

/tmp/ipykernel_20351/3213278412.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_respiratorio.groupby(by=['SEXO','AREA_UR']).size()


SEXO             AREA_UR        
Hombre           No especificada     1240
                 Rural              10362
                 Urbana             50348
Mujer            No especificada      376
                 Rural               7176
                 Urbana             34816
No especificado  No especificada        4
                 Rural                  4
                 Urbana                 8
dtype: int64

Por ejemplo, agrupemos los datos por sexo y área urbana/rural y calculemos el promedio de edad. 

Nota cuidadosamente el orden de las selecciones y operaciones.

In [24]:
df_respiratorio.groupby(by=['SEXO','AREA_UR'])['EDAD'].mean()

/tmp/ipykernel_20351/3408916702.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_respiratorio.groupby(by=['SEXO','AREA_UR'])['EDAD'].mean()


SEXO             AREA_UR        
Hombre           No especificada    49.620265
                 Rural              60.431467
                 Urbana             59.549695
Mujer            No especificada    52.495726
                 Rural              63.834704
                 Urbana             66.127564
No especificado  No especificada          NaN
                 Rural              19.500000
                 Urbana              0.000000
Name: EDAD, dtype: float64

La función `.aggregate()` o `.agg()` se utiliza para aplicar una o varias funciones de agregación a un DataFrame. Esta función es muy útil cuando queremos aplicar diferentes funciones de agregación a diferentes columnas de un DataFrame o cuando queremos aplicar funciones de agregación personalizadas.

Por ejemplo, calculemos varias estadísticas de la `EDAD` para cada categoría de `SEXO`.

In [25]:
df_respiratorio.groupby(by='SEXO')['EDAD'] \
                        .agg(['min', 'max', 'mean', 'median', 'std'])

/tmp/ipykernel_20351/423896262.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_respiratorio.groupby(by='SEXO')['EDAD'] \


,min,max,mean,median,std
SEXO,,,,,
Hombre,0.0,120.0,59.527821,66.0,25.984317
Mujer,0.0,112.0,65.625909,74.0,26.550351
No especificado,0.0,78.0,6.500000,0.0,22.516660


También se pueden aplicar distintas funciones a cada columna usando el siguiente formato.

```
        .agg
            (
                result_col1=(target_col1, function1),
                result_col2=(target_col2, function2),
                result_col3=(target_col2, function2),
            )
```

Estas funciones pueden incluir funciones definidas en `pandas`, de otras bibliotecas o propias.

In [27]:
df_respiratorio.groupby('MES_OCURR').agg(
    Edad_min = ('EDAD','min'),
    Edad_mediana = ('EDAD','median'),
    Edad_max = ('EDAD','max'),
    Derechohab_moda = ('DERECHOHAB', pd.Series.mode ),
    Causa_def_top5 = ('CAUSA_DEF', textos_mas_comunes ),
    
    )

,Edad_min,Edad_mediana,Edad_max,Derechohab_moda,Causa_def_top5
MES_OCURR,,,,,
1.0,0.0,72.0,114.0,IMSS,"J189, U071, J449, J440, C349"
2.0,0.0,71.0,112.0,IMSS,"J189, J449, U071, C349, J440"
3.0,0.0,70.0,112.0,IMSS,"J189, J449, U071, C349, J440"
4.0,0.0,69.0,107.0,IMSS,"J189, J449, C349, J440, U071"
5.0,0.0,69.0,119.0,IMSS,"J189, J449, C349, B208, J440"
6.0,0.0,70.0,112.0,IMSS,"J189, J449, C349, J440, B208"
7.0,0.0,68.0,108.0,IMSS,"J189, J449, C349, B208, J440"
8.0,0.0,68.0,105.0,IMSS,"J189, J449, C349, J440, B208"
9.0,0.0,69.0,109.0,IMSS,"J189, J449, C349, J440, J159"


En caso usar funciones con argumentos es necesario usar una función `lambda` envolviendo la función.

In [28]:
df_respiratorio.groupby('MES_OCURR').agg(
    Causa_def_top10 = ('CAUSA_DEF', lambda s: textos_mas_comunes(s, n=10) ),
    )

,Causa_def_top10
MES_OCURR,
1.0,"J189, U071, J449, J440, C349, J159, B208, J841..."
2.0,"J189, J449, U071, C349, J440, B208, J841, J159..."
3.0,"J189, J449, U071, C349, J440, B208, J159, J841..."
4.0,"J189, J449, C349, J440, U071, B208, J159, J841..."
5.0,"J189, J449, C349, B208, J440, J841, J159, U071..."
6.0,"J189, J449, C349, J440, B208, J159, J841, I269..."
7.0,"J189, J449, C349, B208, J440, J159, J841, P220..."
8.0,"J189, J449, C349, J440, B208, J159, J841, U071..."
9.0,"J189, J449, C349, J440, J159, B208, J841, U071..."


## 5.f Resumen

En esta lección hemos aprendido varios conceptos:

* Tablas pivote: resumen y reorganizan datos
    * Índice: categoría principal
    * Columnas: categoría adicional
    * Valores: datos a resumir
    * Función: función para resumir los datos
    * Nota: se puede usar un multi-índice para incorporar variables
* Funciones útiles: `.mean()`, `.median()`, `.std()`, `.sum()`, `.size()`, `.count()`, `.first()`, `.last()`, `.nth()`, `.min()`, `.max()`

* Melt: tablas pivote a tidy data
    * `id_vars`: Columnas que actúan como identificadores y no deben transformarse.
    * `value_vars`: Columnas que se convertirán en las nuevas columnas de variable y value. Si no se especifica, se utilizan todas las columnas que no están en id_vars.

* Concat: unir tablas uno a uno dependiendo del eje

* Merge: unir tablas muchos a uno
    * Determinar que columna unir en cada tabla
    * En la tabla de catálogo la columna a unir debe de tener valores únicos
    * Existen varias formas de unir tablas: `inner`, `outer`, `left`, `right`

* El patrón División-Aplicación-Combinación (split-apply-combine) permite realizar análisis por subgrupos de forma eficiente y flexible.
    1. Dividir el conjunto de datos en grupos con `groupby()`
    2. Aplicar funciones como agregación, transformación o filtrado
    3. Combinar los resultados en un nuevo DataFrame

* La función agregar `.agg()` permite hacer cálculos resumiendo cada grupo en un parámetro
    * Se pueden usar funciones predefinidas con cadenas de texto, de `pandas`, otras bibliotecas o `lambda`
    * Varias funciones sobre todas las columnas usando una lista: `df.groupby(variables).agg([funcion1, funcion2, ...])`
    * Formato para múltiples columnas y funciones (recuerda que los nombres de columnas van sin comillas):
      ```
        .agg
            (
                result_col1=(target_col1, function1),
                result_col2=(target_col2, function2),
                result_col3=(target_col2, function2),
            )
        ```

**¡Gracias!**